---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less affected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.  Ie,
        remove the [edit].
    2. For "RegionName", when applicable, removing every character from " (" to the end.
        Ie, remove the schools that are in the region.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # Open the file and add rows.  Don't want new line, so filter out last index.
    with open('university_towns.txt', 'r') as file:
        data = []  
        for line in file:
            data.append(line[:-1])  
    
    """
    If a row ends with [edit], it is a state.  Take acronym from states dict
    and make it the state.  If there's an opening bracket in the row, it's a town.
    Else, it's also a town.
    """
    state_region = []
    for row in data:  
        if row[-6:]=='[edit]':  
            state = row[:-6]
            for ac, st in states.items():
                if st == state:
                    state = ac
        elif '(' in row:  
            region = row[:row.index('(')-1]
            state_region.append([state, region]) 
        else:
            region = row
            state_region.append([state, region])
    state_region_df = pd.DataFrame(state_region, columns=['State', 'RegionName'])
    
    return state_region_df

get_list_of_university_towns()

,State,RegionName
0,AL,Auburn
1,AL,Florence
2,AL,Jacksonville
3,AL,Livingston
4,AL,Montevallo
...,...,...
512,WI,River Falls
513,WI,Stevens Point
514,WI,Waukesha
515,WI,Whitewater


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3.  Remember that a recession
    is defined as starting with two consecutive quarters of GDP decline, 
    and ending with two consecutive quarters of GDP growth.'''
    
    gdp = pd.read_excel('gdplev.xls', skiprows=7)
    gdp = gdp[['Unnamed: 4', 'Unnamed: 5']]
    gdp.columns = ['Quarter', 'GDP']
    # Instructions indicate to start at 2000q1.
    Quarter = gdp[gdp['Quarter']=='2000q1'].index
    gdp = gdp.iloc[Quarter[0]:]
    
    for i in range(len(gdp)-2):  
        if (gdp.iloc[i+2,1] < gdp.iloc[i+1,1]) & (gdp.iloc[i+1,1] < gdp.iloc[i,1]): 
            rec_start = gdp.iloc[i,0]
            return rec_start        

get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3.
    
    It is already known that the recession started 2008q3.  Therefore,
    anytime after 2009q1, if there are 2 consecutive quarters
    with growth, that is the end of the recession.'''
    
    gdp = pd.read_excel('gdplev.xls', skiprows=7)
    gdp = gdp[['Unnamed: 4', 'Unnamed: 5']]
    gdp.columns = ['Quarter', 'GDP']
    Start = gdp[gdp['Quarter']=='2008q3'].index
    gdp = gdp.iloc[Start[0]+2:]
    
    for i in range(len(gdp)-2):
        if (gdp.iloc[i+2,1] > gdp.iloc[i+1][1]) & (gdp.iloc[i+1][1] > gdp.iloc[i][1]): 
            rec_end = gdp.iloc[i+2,0]
            return rec_end

get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3.
    
    Find the minimum GDP between the beginning and the end of the recession.'''
    
    gdp = pd.read_excel('gdplev.xls', skiprows=7)
    gdp = gdp[['Unnamed: 4', 'Unnamed: 5']]
    gdp.columns = ['Quarter', 'GDP']
    Start = gdp[gdp['Quarter']=='2008q3'].index
    End = gdp[gdp['Quarter']=='2009q4'].index
    
    gdp = gdp.iloc[Start[0]:End[0]+1]
    Min = str(gdp.loc[gdp['GDP'].idxmin()][0])
    return Min

get_recession_bottom()

'2009q2'

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    
    First thing to do is to rename the indexes and remove the unnecessary
    columns.  Then, groupby the columns and mean the values.
    '''
    
    housing = pd.read_csv('City_Zhvi_AllHomes.csv', encoding = "ISO-8859-1")
    housing = housing.set_index(['State', 'RegionName'])
    housing = housing.loc[:, '2000-01':'2016-09']
    housing = housing.groupby(pd.PeriodIndex(housing.columns, freq='Q'),
                               axis=1).mean().rename(columns=lambda c: str(c).
                                                     lower())
    return housing
        
convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State RegionName                                                   
NY    New York       242860.333333  251626.666667  259494.000000   
CA    Los Angeles    232974.000000  238973.000000  245283.000000   
TX    Houston        106342.666667  106465.666667  107154.666667   
IL    Chicago        154378.000000  159241.666667  163591.666667   
TX    San Antonio    101201.666667  100710.333333   99437.333333   
...                            ...            ...            ...   
MN    Dundee                   NaN            NaN            NaN   
PA    Lewisberry     156017.333333  158735.333333  162760.333333   
      Wellsville     127267.000000  132729.000000  135568.000000   
CA    Challenge                NaN            NaN            NaN   
OH    White Cottage            NaN            NaN            NaN   

                            2000q4         2001q1         2001q2  \
State RegionName                                                   
NY    New York       267818.666667  276433.666667  285866.000000   
CA    Los Angeles    251748.333333  257945.333333  264735.000000   
TX    Houston        108306.333333  108969.333333  108716.333333   
IL    Chicago        168046.000000  172546.333333  177071.000000   
TX    San Antonio     99342.000000   98957.000000   98488.333333   
...                            ...            ...            ...   
MN    Dundee                   NaN            NaN            NaN   
PA    Lewisberry     164300.333333  163497.000000  162813.000000   
      Wellsville     137896.333333  138924.666667  135980.000000   
CA    Challenge                NaN            NaN            NaN   
OH    White Cottage            NaN            NaN            NaN   

                            2001q3         2001q4         2002q1  \
State RegionName                                                   
NY    New York       292638.333333  299623.333333  306852.666667   
CA    Los Angeles    271078.000000  278488.000000  287317.666667   
TX    Houston        108935.000000  109513.333333  110291.333333   
IL    Chicago        181459.666667  185897.666667  189738.666667   
TX    San Antonio     98529.000000   98217.666667   98279.000000   
...                            ...            ...            ...   
MN    Dundee                   NaN            NaN            NaN   
PA    Lewisberry     163148.000000  166082.333333  169035.333333   
      Wellsville     134987.333333  135739.333333  136578.000000   
CA    Challenge                NaN            NaN            NaN   
OH    White Cottage            NaN            NaN            NaN   

                            2002q2  ...         2014q2         2014q3  \
State RegionName                    ...                                 
NY    New York       307169.000000  ...  491350.333333  498070.333333   
CA    Los Angeles    298303.333333  ...  523490.000000  526977.333333   
TX    Houston        111900.333333  ...  140878.000000  145157.333333   
IL    Chicago        193596.000000  ...  194939.333333  198000.666667   
TX    San Antonio     99182.333333  ...  132999.666667  134658.333333   
...                            ...  ...            ...            ...   
MN    Dundee                   NaN  ...   44621.333333   43330.333333   
PA    Lewisberry     170536.333333  ...  209467.333333  210020.333333   
      Wellsville     137517.000000  ...  176005.666667  179007.000000   
CA    Challenge                NaN  ...  169622.666667  170420.333333   
OH    White Cottage            NaN  ...   59662.000000   61631.000000   

                            2014q4         2015q1         2015q2  \
State RegionName                                                   
NY    New York       505276.333333  512158.000000  521124.000000   
CA    Los Angeles    530413.000000  542677.333333  551672.333333   
TX    Houston        149477.000000  153091.333333  156613.666667   
IL    Chicago        200575.333333  202142.666667  205379.000000   
TX    San Anton

In [8]:
def run_ttest():
    '''
    University towns have their mean housing prices less affected by recessions. 
    Run a t-test to compare the ratio of the mean price of houses in university 
    towns the quarter before the recession starts compared to the recession bottom. 
    (price_ratio=quarter_before_recession/recession_bottom)
    
    First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # Retrieve necessary data
    uni_towns = get_list_of_university_towns()
    housing = convert_housing_data_to_quarters()
    r_start_prior = housing.columns.get_loc(get_recession_start()) - 1
    start_prior_q = housing.columns[r_start_prior]
    r_bottom = housing.columns.get_loc(get_recession_bottom())
    r_bottom_q = housing.columns[r_bottom]
    housing['housing_diff'] = housing.iloc[:, r_bottom] - housing.iloc[:, r_start_prior]
    housing2 = housing.iloc[:,[r_start_prior, r_bottom, -1]]
    housing2 = housing2.reset_index()
    
    """
    Price differences for all towns is available.  Need to merge/join the
    the uni dataset with the housing dataset.
    """
    # Get university town dataset.
    uni = pd.merge(uni_towns, housing2, on=['State', 'RegionName'], how='inner')
    uni['uni_town'] = True
    # Get non-university town datasset.
    house = pd.merge(uni, housing2, on=['State', 'RegionName', 
                                        start_prior_q, r_bottom_q,
                                        'housing_diff'], how='right')
    # Replace NaN values with False.
    house = house.replace({'uni_town': {np.NaN: False}})
    uni_series = uni['housing_diff']
    non_uni_series = house[house['uni_town']==False]['housing_diff']
    
    # Do ttest
    t, p = ttest_ind(uni_series.dropna(), non_uni_series.dropna())
    if p < 0.01:
        different = True
    else:
        different = False
        
    # Determine which series has lower mean housing decrease
    if uni_series.mean() < non_uni_series.mean():
        better = 'non-university town'
    else:
        better = 'university town'
    
    return (different, p, better) 

run_ttest()

(False, 0.6154548181208519, 'non-university town')